In [60]:
import pandas as pd
import numpy as np

elas = \
    pd.read_csv("C:\\Users\\amondal\\model_exp\\prev_month_family_values.tsv", sep="\t")

In [61]:
curr_fv = \
    pd.read_csv("C:\\Users\\amondal\\model_exp\\family_values.tsv", sep="\t")

In [62]:
cap_map = \
    pd.read_csv("C:\\Users\\amondal\\model_exp\\cap_map.tsv", sep="\t")

In [63]:
capacity = \
    pd.read_csv("C:\\Users\\amondal\\model_exp\\capacity.tsv", sep="\t")

In [64]:
discount_grps = \
    pd.read_csv("C:\\Users\\amondal\\model_exp\\resources\\discount_groups.tsv", sep="\t")

In [65]:
import json

with open("C:\\Users\\amondal\\model_exp\\resources\\geo_map_prod.json") as f:
    json_str = f.read()
    geo_map = json.loads(json_str)

In [66]:
with open("C:\\Users\\amondal\\model_exp\\resources\\sku_threshold_map.json") as f:
    json_str = f.read()
    sku_threshold = json.loads(json_str)

In [67]:
with open("C:\\Users\\amondal\\model_exp\\resources\\sku_type_map.json") as f:
    json_str = f.read()
    sku_type = json.loads(json_str)

In [68]:
with open("C:\\Users\\amondal\\model_exp\\resources\\price_constraints.json") as f:
    json_str = f.read()
    price_constraints = json.loads(json_str)

In [69]:
grps = discount_grps['discount_group'].to_list()
grps = [eval(x) for x in grps]
grp_id_map = {}
for i in range(len(grps)):
    for x in grps[i]:
        grp_id_map[x] = i

In [70]:
elas['dg'] = elas['pre_process_family'].apply(lambda x: grp_id_map[x])

In [71]:
elas['alpha'] = np.nan
elas['alpha'] = \
    (elas['OS'] == 'windows')*0.68 + \
    (elas['OS'] == 'linux')*1.06

In [72]:
elas['beta'] = np.nan
elas['beta'] = \
    (elas['OS'] == 'windows')*-1.97 + \
    (elas['OS'] == 'linux')*-2.19

In [73]:
elas['gamma'] = np.nan
elas['gamma'] = \
    (elas['OS'] == 'windows')*0.0 + \
    (elas['OS'] == 'linux')*0.0019

In [74]:
elas['factor'] = elas['spot_core_price'].pow(elas['alpha']*elas['weight'])

In [75]:
factors = \
    elas[['pre_process_family', 'OS', 'factor']]\
    .groupby(by=['pre_process_family', 'OS'],as_index=False)\
    .agg(prod_factors=('factor', 'prod'))

elas = pd.merge(elas, factors, on=['pre_process_family', 'OS'], how='left')

In [76]:
elas['demand_without_k'] = \
    elas['spot_core_price'].pow(elas['beta']) * elas['prod_factors'] / elas['factor']

In [77]:
elas['avg_daily_demand_w_nulls'] = \
    elas['totalDemand_ext_spot_nofraud'] / elas['totalDays_ext_spot_nofraud']

elas['avg_daily_demand_removed_nulls'] = \
    elas['avg_daily_demand_w_nulls'].fillna(0.0)

In [78]:
elas['od_avg_daily_demand_w_nulls'] = \
    elas['totalDemand_ext_od_nofraud'] / elas['totalDays_ext_od_nofraud']

elas['od_avg_daily_demand_removed_nulls'] = \
    elas['od_avg_daily_demand_w_nulls'].fillna(0.0)

In [79]:
elas['k'] = elas['avg_daily_demand_removed_nulls']/elas['demand_without_k']

In [80]:
elas[elas.k.isna()]

,Location,pre_process_family,OS,weight_check,spot_core_price,od_core_price,current_avg_discount,min_spot_core_price,max_spot_core_price,min_od_core_price,...,beta,gamma,factor,prod_factors,demand_without_k,avg_daily_demand_w_nulls,avg_daily_demand_removed_nulls,od_avg_daily_demand_w_nulls,od_avg_daily_demand_removed_nulls,k


In [81]:
override_columns = list(set(curr_fv.columns) - {"Location", "pre_process_family", "OS"})

curr_fv = curr_fv.rename(
    columns={
        col : f"{col}_pre_override"\
        for col in override_columns
    }
)

elas = pd.merge(elas, 
                curr_fv, 
                on=['Location', 'pre_process_family', 'OS'], 
                how="inner")

In [82]:
elas['od_core_price'].to_list()[:10]

[0.13335,
 0.1014,
 0.0678285714285714,
 0.1153749999999999,
 0.104625,
 0.1506249999999999,
 0.1353125,
 0.1813125,
 0.082494140625,
 0.128494140625]

In [83]:
elas['od_core_price'] = \
    ( 1.0 / ( 1.0 - elas['current_avg_discount'] ) ) * elas['spot_core_price']

elas['int_rounded_discount'] = \
    (100 * elas['current_avg_discount'].apply(lambda x : round(x, 2) )).apply(int)


In [84]:
elas['od_core_price'].to_list()[:10]

[0.13335014175578816,
 0.10140209396610887,
 0.06782857142857095,
 0.1153749999999999,
 0.10462500000000001,
 0.15062500360536585,
 0.13531250000000003,
 0.1813125007496699,
 0.08249414062499996,
 0.12849414062499998]

In [85]:
elas['int_rounded_discount'].to_list()[:10]

[54, 32, 90, 90, 90, 90, 90, 90, 68, 68]

In [86]:
k_vals =\
    elas[['Location', 'dg', 'OS', 'k']]\
    .groupby(['Location', 'dg', 'OS'], as_index=False)\
    .max()

k_vals['opt'] = k_vals['k'] < 0.00001

elas = \
    pd.merge(\
        elas,
        k_vals[['Location', 'dg', 'OS', 'opt']],
        how='inner',
        on = ['Location', 'dg', 'OS']
    )

In [87]:
elas_dg_er = \
    elas[['Location', 'dg', 'Evictions_FirstHourEvictedCores', 'Evictions_TotalCores']]\
    .groupby(by=['Location', 'dg'], as_index=False)\
    .agg(\
        Evictions_FirstHourEvictedCores_dg=('Evictions_FirstHourEvictedCores', 'sum'),
        Evictions_TotalCores_dg=('Evictions_TotalCores', 'sum'))

elas_dg_er['dg_er'] = \
    (elas_dg_er['Evictions_FirstHourEvictedCores_dg'])/(elas_dg_er['Evictions_TotalCores_dg']+1)

elas_dg_er['dg_high'] = elas_dg_er['dg_er'] >= 0.05

elas = pd.merge(elas, 
                elas_dg_er, 
                on=['Location', 'dg'], 
                how='inner')

In [88]:
elas['family_er'] = \
    (elas['Evictions_FirstHourEvictedCores'])/(elas['Evictions_TotalCores']+1)

elas['family_high'] = elas['family_er'] > 0.05

In [89]:
df5 = elas[elas.family_high != elas.dg_high][['Location', 'pre_process_family', 'OS', 'family_er', 'dg_er', 'family_high', 'dg_high']]
df5.to_csv("conflict_dg_family_er.csv", sep=",")

In [90]:
elas_dg_demand = \
    elas[['Location', 'dg', 'avg_daily_demand_removed_nulls']]\
    .groupby(by=['Location', 'dg'],as_index=False)\
    .agg(\
        dg_total_demand=('avg_daily_demand_removed_nulls', 'sum'))

elas = pd.merge(elas, 
                elas_dg_demand, 
                on=['Location', 'dg'], 
                how='inner')

In [91]:
elas['min_price_er'] = \
    (elas['dg_high'] * 0.9 * elas['spot_core_price']) + \
    (~elas['dg_high'] * 0.8 * elas['spot_core_price']) 

In [92]:
elas['max_price_er'] = \
    (elas['dg_high'] * 1.1 * elas['spot_core_price']) + \
    (~elas['dg_high'] * 1.05 * elas['spot_core_price']) 

In [93]:
elas['min_price_global'] = 0.1*elas['od_core_price']

In [94]:
elas['max_price_global'] = 0.7*elas['od_core_price']

In [95]:
elas[['min_price_er', 'max_price_er', 'min_price_global', 'max_price_global']]

,min_price_er,max_price_er,min_price_global,max_price_global
0,0.049019,0.064338,0.013335,0.093345
1,0.055070,0.072279,0.010140,0.070981
2,0.005426,0.007122,0.006783,0.047480
3,0.007377,0.009683,0.009182,0.064275
4,0.007377,0.009683,0.009182,0.064275
...,...,...,...,...
7986,0.120266,0.157849,0.060133,0.420931
7987,0.115061,0.151017,0.059965,0.419755
7988,0.119055,0.156260,0.064789,0.453520
7989,0.124415,0.163294,0.064584,0.452088


In [96]:
elas['min_price'] = elas[["min_price_er", "min_price_global"]].max(axis=1)

In [97]:
low_demand = elas['dg_total_demand'] <= 5000.0

elas['max_price'] = \
    low_demand*elas[["spot_core_price", "min_price"]].max(axis=1) + \
    ~low_demand*elas[["max_price_er", "max_price_global"]].min(axis=1)

small_diff = (elas['max_price'] - elas['min_price']).abs() <= 0.00001
elas['max_price'] = small_diff*elas['min_price'] + ~small_diff*elas['max_price']

In [98]:
elas[['min_price', 'max_price']]

,min_price,max_price
0,0.049019,0.061274
1,0.055070,0.068837
2,0.006783,0.006783
3,0.009182,0.009221
4,0.009182,0.009221
...,...,...
7986,0.120266,0.150333
7987,0.115061,0.143826
7988,0.119055,0.148819
7989,0.124415,0.155518


In [99]:

elas['max_feasible_family_discount'] = \
    1.0 - ( elas['min_price'] / elas['od_core_price'] )

elas['min_feasible_family_discount'] = \
    1.0 - ( elas['max_price'] / elas['od_core_price'] )

elas['family_discount'] = elas['current_avg_discount']

dg_agg_1 =\
    elas\
        [['Location', 'dg', 'pre_process_family', 'avg_daily_demand_removed_nulls']]\
        .groupby(by=['Location', 'dg', 'pre_process_family'], as_index=False)\
        .sum()

dg_agg_2 =\
    dg_agg_1\
        [['Location', 'dg', 'avg_daily_demand_removed_nulls']]\
        .groupby(by=['Location', 'dg'], as_index=False)\
        .sum()\
        .rename(columns = {"avg_daily_demand_removed_nulls" : "total_daily_demand"})

elas = pd.merge(elas, dg_agg_2, how = 'inner', on = ['Location', 'dg'])

elas['family_prop_of_total_dg_demand'] = \
    elas['avg_daily_demand_removed_nulls'] / elas['total_daily_demand']

elas['family_prop_of_total_dg_demand'] = \
    elas['family_prop_of_total_dg_demand'].fillna(0.0)

In [100]:

current_discount_group_bounds =\
    elas[['Location', 'dg', 'OS', 'int_rounded_discount', 'min_feasible_family_discount', 'max_feasible_family_discount']]\
        .groupby(['Location', 'dg', 'OS', 'int_rounded_discount'], as_index=False)\
        .agg(
            {
                'max_feasible_family_discount' : 'min',
                'min_feasible_family_discount' : 'max',
            }
        )

elas =\
    elas\
    .rename(
        columns={
            'max_feasible_family_discount' : 'max_feasible_family_discount_pre_limit',
            'min_feasible_family_discount' : 'min_feasible_family_discount_pre_limit',
        }
    )\
    .merge(
        current_discount_group_bounds,
        how='inner',
        on = ['Location', 'dg', 'OS', 'int_rounded_discount']
    )

In [101]:
wm = lambda vec: np.average(vec, weights= (elas.loc[vec.index, "family_prop_of_total_dg_demand"] + 0.00001))

dg_elas = \
    elas\
    [['Location', 'OS', 'dg', 'max_feasible_family_discount', 'min_feasible_family_discount', 'family_discount']]\
    .groupby(['Location', 'OS', 'dg'], as_index=False)\
    .agg(
        {
            'max_feasible_family_discount' : 'min',
            'min_feasible_family_discount' : 'max',
            'family_discount' : wm
        }
    )\
    .rename(
        columns={
            'max_feasible_family_discount' : 'max_feasible_dg_discount',
            'min_feasible_family_discount' : 'min_feasible_dg_discount',
            'family_discount' : 'dg_discount'
        }
    )

elas = \
    pd.merge(\
        elas, 
        dg_elas, 
        how='inner',
        on = ['Location', 'OS', 'dg']\
    )

In [102]:

elas['dg_satisfiable'] = \
    elas['min_feasible_dg_discount'] < elas['max_feasible_dg_discount']

desired_discount_below_bounds = \
    ( elas['min_feasible_family_discount'] > elas['dg_discount'] )

desired_discount_above_bounds = \
    ( elas['max_feasible_family_discount'] < elas['dg_discount'] )

desired_discount_oob =  \
    ( ( desired_discount_above_bounds ) | ( desired_discount_below_bounds ) )

elas['fixed_discount'] = \
    desired_discount_oob*0.0 + ~desired_discount_oob*elas['dg_discount']

elas['fixed_discount'] = \
    desired_discount_below_bounds*elas['min_feasible_family_discount'] + \
    desired_discount_above_bounds*elas['max_feasible_family_discount']

elas['min_price'] = \
    elas['dg_satisfiable']*elas['min_price'] + \
    ~elas['dg_satisfiable']*(( ( 1.0 - elas['fixed_discount'] ) * elas['od_core_price'] ))

elas['max_price'] = \
    elas['dg_satisfiable']*elas['max_price'] + \
    ~elas['dg_satisfiable']*( ( 1.0 - elas['fixed_discount'] ) * elas['od_core_price'] )


In [103]:
compete_fields = ['pre_process_family', 'OS']

_, compete_group_ids = np.unique(
    elas[compete_fields].apply(lambda x : '|'.join(x), axis=1).to_numpy(),
    return_inverse=True
)

elas['group'] = compete_group_ids

In [104]:
elas['group']

0         4
1         5
2         2
3        32
4        36
       ... 
7986    198
7987    199
7988    209
7989    200
7990    210
Name: group, Length: 7991, dtype: int64

In [105]:
elas['alpha_original'] = elas['alpha'].copy()
elas['beta_original'] = elas['beta'].copy()
elas['k_original'] = elas['k'].copy()
elas['weight_original'] = elas['weight'].copy()

elas['alpha'] = elas['opt']*elas["alpha"] + ~elas['opt']*0.0
elas['beta'] = elas['opt']*elas["beta"] + ~elas['opt']*0.0
elas['weight'] = elas['opt']*elas["weight"] + ~elas['opt']*0.0
elas['k'] = elas['opt']*elas["k"] + ~elas['opt']*0.1

In [106]:
def segment_sum(arr, segments):
    m = {}
    for i in range(len(segments)):
        if segments[i] not in m:
            m[segments[i]] = 0.0
            
        m[segments[i]] += arr[i]

    return np.array([m[s] for s in segments])

def demand_function(
    p: np.ndarray,
    k: np.ndarray,
    alpha: np.ndarray,
    beta: np.ndarray,
    weight: np.ndarray,
    group: np.ndarray,
    n_groups: int,
) -> np.ndarray:
    """
    @param p: price (vector)
    @param k: demand scaler (vector)
    @param alpha: alpha parameter (float or vector)
    @param beta: beta parameter (float or vector)
    @param weight: "weight" (float or vector)
    """
    return \
        (np.exp(segment_sum(np.log(p) * (alpha * weight), group))
        * k
        * np.power(p, beta - alpha * weight)
    )

In [ ]:
results = []
print("Computing Min/Max Demand")

for group in list(elas['group'].unique()):
    restr = elas[elas['group'] == group].copy().reset_index(drop=True)

    tasks = list(
        restr[['pre_process_family', 'OS', 'Location']].drop_duplicates().apply(
            lambda x : {
                'pre_process_family' : x['pre_process_family'],
                'OS' : x['OS'],
                'Location' : x['Location']
            },
            axis=1
        )
    )

    for task in tasks:
        sub_restr = restr.copy()
        mask =  ((sub_restr['pre_process_family'] == task['pre_process_family']) & (sub_restr['Location'] == task['Location']) & (sub_restr['OS'] == task['OS']))

        sub_restr['p_min_demand'] = \
            mask*sub_restr['max_price'] + ~mask*sub_restr['min_price']

        sub_restr['p_max_demand'] = \
            mask*sub_restr['min_price'] + ~mask*sub_restr['max_price']

        sub_restr['group'] = 0

        sub_restr['min_demand'] = demand_function(
            p=sub_restr['p_min_demand'].to_numpy(),
            k=sub_restr['k'].to_numpy(),
            alpha=sub_restr['alpha'].to_numpy(),
            beta=sub_restr['beta'].to_numpy(),
            weight=sub_restr['weight'].to_numpy(),
            group=sub_restr['group'].to_numpy(),
            n_groups=1
        )

        sub_restr['max_demand'] = demand_function(
            p=sub_restr['p_max_demand'].to_numpy(),
            k=sub_restr['k'].to_numpy(),
            alpha=sub_restr['alpha'].to_numpy(),
            beta=sub_restr['beta'].to_numpy(),
            weight=sub_restr['weight'].to_numpy(),
            group=sub_restr['group'].to_numpy(),
            n_groups=1
        )

        results.append(
            {
                'pre_process_family' : task['pre_process_family'],
                'Location' : task['Location'],
                'OS' : task['OS'],
                'min_demand' : sub_restr[mask]['min_demand'].iloc[0],
                'max_demand' : sub_restr[mask]['max_demand'].iloc[0] 
            }
        )
        
print("Computed Min/Max Demand")

append_df = pd.DataFrame(data=results)
elas = \
    pd.merge(\
        elas,
        append_df,
        how='inner',
        on=['pre_process_family', 'Location', 'OS']
    )

elas['model_view_demand_pre_override'] = demand_function(
    p=elas['spot_core_price_pre_override'].to_numpy(),
    k=elas['k'].to_numpy(),
    alpha=elas['alpha'].to_numpy(),
    beta=elas['beta'].to_numpy(),
    weight=elas['weight'].to_numpy(),
    group=elas['group'].to_numpy(),
    n_groups=elas['group'].max() + 1
)

elas['model_view_demand_post_override'] = demand_function(
    p=elas['spot_core_price'].to_numpy(),
    k=elas['k'].to_numpy(),
    alpha=elas['alpha'].to_numpy(),
    beta=elas['beta'].to_numpy(),
    weight=elas['weight'].to_numpy(),
    group=elas['group'].to_numpy(),
    n_groups=elas['group'].max() + 1
)


In [110]:
demand = demand_function(
    p=elas['spot_core_price_pre_override'].to_numpy(),
    k=elas['k_original'].to_numpy(),
    alpha=elas['alpha_original'].to_numpy(),
    beta=elas['beta_original'].to_numpy(),
    weight=elas['weight_original'].to_numpy(),
    group=elas['group'].to_numpy(),
    n_groups=elas['group'].max() + 1
)

In [111]:
a = list(demand)
b = elas['avg_daily_demand_removed_nulls'].to_list()

In [114]:

d = \
    elas["k_original"]*elas["spot_core_price_pre_override"].pow(elas['beta_original'])*elas["prod_factors"]/elas["factor"]

In [116]:
d = d.to_list()
print(list(zip(a, b, d)))

[(0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (88.3688698163729, 96.25937026666666, 96.25937026666668), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (195.14430076352627, 192.50318466666664, 192.50318466666667), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (45.419072219395204, 48.12624173846154, 48.12624173846154), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.

In [ ]:
print(list(zip(a, b)))

In [113]:
assert np.allclose(demand, elas['avg_daily_demand_removed_nulls'])

AssertionError: 